In [218]:
# importing libraries
import pandas as pd
from datetime import datetime
import numpy as np
import math
import ast

In [219]:
df = pd.read_csv(r'Data\IMDB_cleaned.csv')

In [220]:
df.sample(20)

,tconst,titleType,primaryTitle,originalTitle,startYear,genres,averageRating,numVotes
407350,tt0949412,short,Gosunkugi Torakichi,Gosunkugi Torakichi,1912,Short,6.8,66
1788930,tt8199532,short,Are You Gonna Eat That?,Are You Gonna Eat That?,2017,"Comedy, Short",6.7,669
538031,tt11585292,short,Sound the Bugle,Sound the Bugle,2018,"Music, Short",6.8,385
1403668,tt4058858,movie,Marciano,Marciano,2014,"Documentary, Drama, Family",6.5,2416
1168304,tt23577152,series,Aaina (Truth Lies Within),Aaina (Truth Lies Within),2022,"Drama, Romance",9.3,23
1457312,tt4560750,short,Dull Moments,En tam forestilling,2015,"Comedy, Drama, Short",6.7,930
1444273,tt4446506,short,"Shark, O tubarão amigo","Shark, O tubarão amigo",2006,"Animation, Short",6.8,603
1880062,tt9559986,short,Le città tardo barocche del Val di Noto,Le città tardo barocche del Val di Noto,2018,"Documentary, Short",6.9,113
411484,tt0974544,movie,Coming Down the Mountain,Coming Down the Mountain,2007,Drama,7.4,518
255126,tt0336728,short,Lard Refinery,Lard Refinery,1901,"Documentary, Short",6.9,113


In [221]:
df['genres'] = df['genres'].apply(lambda x: x.replace(" ", "").lower().split(","))

In [222]:
df.head()

,tconst,titleType,primaryTitle,originalTitle,startYear,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,1894,"[documentary, short]",5.7,1954
1,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,1892,"[animation, comedy, romance]",6.5,1787
2,tt0000004,short,Un bon bock,Un bon bock,1892,"[animation, short]",5.6,179
3,tt0000005,short,Blacksmith Scene,Blacksmith Scene,1893,"[comedy, short]",6.2,2589
4,tt0000006,short,Chinese Opium Den,Chinese Opium Den,1894,[short],5.1,177


In [223]:
# creating a dict file containing movie id as key and date and rating as value in a tuple

test_dict = {"tt0341281": ("20/05/2023", "6"), "tt6376792": ("21/05/2022", "7"),
            "tt1022045": ("22/03/2023", "4"), "tt0531025": ("23/04/2023", "7"),
            "tt2289228	": ("24/05/2023", "9"), "tt1994702": ("25/05/2023", "8"),
            "tt0036775": ("26/05/2023", "7"), "tt0836868": ("27/05/2023", "6"),
            "tt0038355": ("28/05/2023", "5"), "tt8123920": ("29/05/2023", "4"),
            "tt9449962": ("30/05/2023", "9"), "tt2297810": ("31/05/2023", "2"),
            "tt9856280": ("01/06/2019", "6"), "tt0640522": ("02/06/2023", "6"),
            "tt7040746": ("03/06/2023", "3"), "tt0040897": ("04/06/2023", "4"),
            "tt8041959": ("05/06/2023", "5"), "tt8041959": ("06/06/2023", "6"),
            "tt8042041": ("07/06/2023", "7"), "tt2640252": ("08/06/2023", "8"),
            "tt8943014": ("09/06/2023", "9"), "tt8044079": ("10/06/2023", "10"),
            "tt8045152": ("11/05/2023", "9"), "tt0046250": ("12/05/2023", "8"),
            "tt0973783": ("15/02/2010", "5"), "tt0047396": ("16/02/2023", "4"),
            "tt16085542": ("17/03/2023", "8"), "tt0347528": ("18/03/2023", "5"),
            "tt0047673": ("19/04/2023", "1"), "tt0947834": ("20/04/2023", "2"),
            "tt1048021": ("21/04/2023", "3"), "tt16138306": ("22/04/2023", "4"),
            }

In [224]:
# dropping all movies that were rated 3 and below
delete_list = []
for key, value in test_dict.items():
    if int(value[1]) <= 3:
        delete_list.append(key)

for key in delete_list:
    del test_dict[key]

In [225]:
# creating a new df to store the required columns
df_input = df.loc[df["tconst"].isin(list(test_dict.keys()))].copy()
df_input.reset_index(inplace=True, drop=True)
df_input["relevance"] = 0

In [226]:
# this function decides on the relevance of each movie based on the date and rating

def relevance(test_dict):
    for key, value in test_dict.items():
        date = datetime.strptime(value[0], "%d/%m/%Y")
        rating = value[1]
        relevance = float(rating) /(math.sqrt((abs(int((date - date.today()).days))))+ 0.01)
        df_input["relevance"].loc[df_input["tconst"] == key] = relevance

In [227]:
relevance(test_dict)

C:\Users\Finn\AppData\Local\Temp\ipykernel_16360\3968286732.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input["relevance"].loc[df_input["tconst"] == key] = relevance


In [228]:

# optaining unique genres and creating a dictionary to count the number of movies in each genre
# adding the relevance of each movie to the corresponding genre
genre_list = []
for i in range(len(df_input)):
    for genre in list(df_input["genres"].iloc[i]):
        genre_list.append(genre)
genre_dict = {genre: 0 for genre in set(genre_list)}

for i in range(len(df_input)):
    temp_list = [df_input["genres"].iloc[i]][0]
    for genre in temp_list:
            genre_dict[genre] += df_input["relevance"].iloc[i]

In [229]:
# converting genre to lowercase and splitting the string into a list
# in order to process rows with multiple genres
df_input["titleType"] = df_input["titleType"].apply(lambda x: str(x).lower())

In [230]:
# optaining unique genres and creating a dictionary to count the number of movies in each genre
# adding the relevance of each movie to the corresponding genre
type_list = df_input["titleType"].tolist()
type_dict = {ttype: 0 for ttype in set(type_list)}

for i in range(len(df_input)):
    temp = [df_input["titleType"].iloc[i]][0]
    type_dict[temp] += df_input["relevance"].iloc[i]

In [231]:
# creating a dictionary for each decade and initializing the value to 0
decade_dict = {"1880-1889" : 0, "1890-1899" : 0, "1900-1909" : 0,"1910-1919" : 0, "1920-1929" : 0,
               "1930-1939" : 0, "1940-1949" : 0, "1950-1959" : 0,"1960-1969" : 0, "1970-1979" : 0,
               "1980-1989" : 0, "1990-1999" : 0, "2000-2009" : 0,"2010-2019" : 0, "2020-2029" : 0
               }

In [232]:
# adding the relevance of each movie to the corresponding decade
for i in range(len(df_input)):
    year = [df_input["startYear"].iloc[i]][0]
    for decade in decade_dict:
        start_year, end_year = map(int, decade.split('-'))
        if start_year <= int(year) <= end_year:
            decade_dict[decade] += df_input["relevance"].iloc[i]

In [233]:
# creating a rating interval and removeing rating with too little votes 
for i in range(len(df_input)):
    if df_input["numVotes"].iloc[i] <= 100:
        df_input["averageRating"].iloc[i] = np.nan

In [234]:
# creating a dictionary for each rating and initializing the value to 0
df_rating = df_input.dropna(subset=["averageRating"])
rating_list = df_rating["averageRating"].tolist()
rating_dict = {rating: 0 for rating in set(rating_list)}

# adding the relevance of each movie to the corresponding rating
for i in range(len(rating_dict)):
    rating = [df_rating["averageRating"].iloc[i]][0]
    rating_dict[rating] += df_input["relevance"].iloc[i]

# creating a weighted mean for each rating
weighted_mean = sum([value*rating for rating,
                     value in rating_dict.items()]) / sum(rating_dict.values())

In [235]:
# filtering the df based on the rating range and minimum number of votes

df = df[df["numVotes"] > 100]
df = df[df["averageRating"].between(round(weighted_mean,1)-0.5,
                                                   round(weighted_mean,1)+0.5, inclusive=True)]

C:\Users\Finn\AppData\Local\Temp\ipykernel_16360\3660041891.py:4: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df = df[df["averageRating"].between(round(weighted_mean,1)-0.5,


In [236]:
# creating a dictionary for each genre and initializing the value to 0
# assigning a value based on the relevance of each genre
genre_values_dict = {}
y = 3
for i in range(3):
    genre_values_dict[max(genre_dict, key = genre_dict.get)] = y
    genre_dict.pop(max(genre_dict, key = genre_dict.get))
    y -= 1

In [237]:
# creating a dictionary for each type and initializing the value to 0
# assigning a value based on the relevance of each type

type_values_dict = {}
y = 3
for i in range(len(type_dict)):
    type_values_dict[max(type_dict, key=type_dict.get)] = y
    type_dict.pop(max(type_dict, key=type_dict.get))
    y -= 1

In [238]:
# creating a dictionary for each decade and initializing the value to 0
# assigning a value based on the relevance of each type
decade_value_dict = {}
decade_value_dict[max(decade_dict, key = decade_dict.get)] = 3
temp_list = list(decade_dict)
decade_value_dict[temp_list[temp_list.index(max(decade_dict, key = decade_dict.get))-1]] = 1
decade_value_dict[temp_list[temp_list.index(max(decade_dict, key = decade_dict.get))+1]] = 1

In [239]:
df.reset_index(inplace=True, drop=True)

In [240]:
# assigning a score to each movie based on the genre, type and decade

df["Genre_Score"] = 0
df["Type_Score"] = 0
df["Decade_Score"] = 0

# assigning a score for the genre(s)
for i in range(len(df)):
    temp = df.iloc[i]["genres"]
    for genre in temp:
        genre = str(genre).lower()
        if genre in genre_values_dict.keys():
            df.at[i, "Genre_Score"] += genre_values_dict[genre]

# assigning as score for the type
    temp = df.iloc[i]["titleType"]
    temp = str(temp).lower()
    if temp in type_values_dict.keys():
        df.at[i,"Type_Score"] += type_values_dict[temp]
    
# assigning a score for the decade/year
    year = df.iloc[i]["startYear"]
    for decade in decade_value_dict:
        start_year, end_year = map(int, decade.split('-'))
        if start_year <= int(year) <= end_year:
            df.at[i,"Decade_Score"] += decade_value_dict[decade]

# calculating the total score
df["Total_Score"] = df["Genre_Score"] + df["Type_Score"] + df["Decade_Score"]

In [241]:
# selecting the top 5 movies based on the total score
df_best_score = df[df.Total_Score == df["Total_Score"].max()]
try:
    recommended = df_best_score.sample(5)[["primaryTitle", "startYear"]]
    output = str()
    for i in range(len(recommended)):
        output= output + (f'{recommended.iloc[i]["primaryTitle"]}({recommended.iloc[i]["startYear"]}), ')
        if i == len(recommended)-1:
            output = output[:-2]
    print(f'The following movies were found based on your preferences: {output}')
except ValueError:
    recommended = df_best_score[["primaryTitle", "startYear"]]
    output = str()
    for i in range(len(recommended)):
        output = output + (f'{recommended.iloc[i]["primaryTitle"]}({recommended.iloc[i]["startYear"]}), ')
        if i == len(recommended)-1:
            output = output[:-2]
    print(f'The following movies were found based on your preferences: {output}')

The following movies were found based on your preferences: A Man Called Ove(2015), Final Cut: Ladies and Gentlemen(2012), Engeyum Eppodhum(2011), The Artist(2011), Ala Modalaindi(2011)


In [242]:
#+# selecting the top 5 movies based on the favorite type and highest score
df_fav_type = df[df.titleType == list(type_values_dict.keys())[0]]
df_fav_type = df_fav_type[df_fav_type.Total_Score == df_fav_type["Total_Score"].max()]
try:
    recommended = df_fav_type.sample(5)[["primaryTitle", "startYear"]]
    output = str()
    for i in range(len(recommended)):
        output= output + (f'{recommended.iloc[i]["primaryTitle"]}({recommended.iloc[i]["startYear"]}), ')
        if i == len(recommended)-1:
            output = output[:-2]
    print(f'The following movies were found based on your favorite Type: {output}')
except ValueError:
    recommended = df_fav_type[["primaryTitle", "startYear"]]
    output = str()
    for i in range(len(recommended)):
        output= output + (f'{recommended.iloc[i]["primaryTitle"]}({recommended.iloc[i]["startYear"]}), ')
        if i == len(recommended)-1:
            output = output[:-2]
    print(f'The following movies were found based on your favorite type: {output}')

The following movies were found based on your favorite Type: Ala Modalaindi(2011), A Man Called Ove(2015), Bojhena Shey Bojhena(2012), Barfi!(2012), Silver Linings Playbook(2012)


In [243]:
# selecting the top 5 movies based on the favorite genre and highest score
df["single_genre"] = df["genres"].apply(lambda x: x[0])
df_fav_genre = df[df.single_genre == list(genre_values_dict.keys())[0]]
df_fav_genre = df_fav_genre[df_fav_genre.Total_Score == df_fav_genre["Total_Score"].max()]

try:
    recommended = df_fav_genre.sample(5)[["primaryTitle", "startYear"]]
    output = str()
    for i in range(len(recommended)):
        output = output + (f'{recommended.iloc[i]["primaryTitle"]}({recommended.iloc[i]["startYear"]}), ')
        if i == len(recommended)-1:
            output = output[:-2]
    print(f'The following movies were found based on your favorite genre: {output}')
except ValueError:
    recommended = df_fav_genre[["primaryTitle", "startYear"]]
    output = str()
    for i in range(len(recommended)):
        output = output + (f'{recommended.iloc[i]["primaryTitle"]}({recommended.iloc[i]["startYear"]}), ')
        if i == len(recommended)-1:
            output = output[:-2]
    print(f'The following movies were found based on your favorite genre: {output}')
    

The following movies were found based on your favorite genre: The King and I(2018), The Sinking of Van Der Wijck(2013), Swapnajaal(2018), Thou Gild'st the Even(2013), The Broken Circle Breakdown(2012)
